# Training Faster-RCNN Model

In [1]:
import sys
sys.path.insert(0,'/workspaces/breast-tumor-detection/Model/FasterRCNN/src/')

In [2]:
import os
import random
import imageio

image_dir = '/workspaces/breast-tumor-detection/Data/Original_Data/pos'
images = os.listdir(image_dir)
select_images = random.sample(images, k = 304)
random.shuffle(select_images)
training_images = select_images[:250]
validation_images = select_images[250:]
#store training and validation images
train_destination_dir = '/workspaces/breast-tumor-detection/Model/FasterRCNN/data/train'
val_destination_dir = '/workspaces/breast-tumor-detection/Model/FasterRCNN/data/val'

#iterate to store
for img_path in training_images:
    img_path_full = os.path.join(image_dir, img_path)
    assert os.path.exists(img_path_full)
    img = imageio.v2.imread(img_path_full)
    imageio.imwrite(os.path.join(train_destination_dir, img_path), img)
#iterate to store
for img_path in validation_images:
    img_path_full = os.path.join(image_dir, img_path)
    assert os.path.exists(img_path_full)
    img = imageio.v2.imread(img_path_full)
    imageio.imwrite(os.path.join(val_destination_dir, img_path), img)

In [3]:
from config import DEVICE, NUM_CLASSES, NUM_EPOCHS, OUT_DIR
from config import VISUALIZE_TRANSFORMED_IMAGES
from config import SAVE_PLOTS_EPOCH, SAVE_MODEL_EPOCH
from model_modified import create_model, MyModel
from utils import Averager, show_tranformed_image
from tqdm.auto import tqdm
from dataset import train_loader, valid_loader
import torch
import matplotlib.pyplot as plt
import time

/workspaces/breast-tumor-detection/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of training samples: 254
Number of validation samples: 61



In [4]:
# function for running training iterations
def train(train_data_loader, model, optimizer, train_itr, train_loss_list, train_loss_hist):
    """Run Training Iterations"""
    print("Training")
    # initialize tqdm progress bar
    prog_bar = tqdm(train_data_loader, total=len(train_data_loader))

    for data in prog_bar:
        optimizer.zero_grad()
        images, targets = data
        images = [image.to(DEVICE) for image in images]
        targets = [
            {k: v.to(DEVICE, dtype=torch.int64) for k, v in t.items()} for t in targets
        ]

        loss_dict = model(images, targets)

        # losses = sum(loss for loss in loss_dict.values())
        losses = loss_dict['loss_box_reg']
        loss_value = losses.item()
        train_loss_list.append(loss_value)

        train_loss_hist.send(loss_value)

        losses.backward(retain_graph=True)
        optimizer.step()

        train_itr += 1

        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return train_loss_list


# function for running validation iterations
def validate(valid_data_loader, model, val_itr, val_loss_list, val_loss_hist):
    """Run Validation Iterations"""
    print("Validating")
    # initialize tqdm progress bar
    prog_bar = tqdm(valid_data_loader, total=len(valid_data_loader))

    for data in prog_bar:
        images, targets = data
        images = [image.to(DEVICE) for image in images]
        targets = [
            {k: v.to(DEVICE, dtype=torch.int64) for k, v in t.items()} for t in targets
        ]
        with torch.no_grad():
            loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        val_loss_list.append(loss_value)
        val_loss_hist.send(loss_value)
        val_itr += 1
        # update the loss value beside the progress bar for each iteration
        prog_bar.set_description(desc=f"Loss: {loss_value:.4f}")
    return val_loss_list

### Initialize model and parameters

In [2]:
import torch

In [3]:
fill_kernel = torch.Tensor([
            1,
            1,
            1,
            1,
            1,
            2 / 3,
            2 / 3,
            2 / 3,
            2 / 3,
            2 / 3,
            1 / 3,
            1 / 3,
            1 / 3,
            1 / 3,
            1 / 3,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
        ])

kernel_size = 30
kernel_north = (
            torch.zeros((1, 1, kernel_size, kernel_size)).type(
                torch.FloatTensor
))

In [16]:
fill_kernel = torch.Tensor([1,1/2,1/3,0,0])
kernel_size = 5
kernel_northeast = (
            torch.zeros((1, 1, kernel_size, kernel_size)).type(
                torch.FloatTensor
))

In [24]:
range(-1,-31)

range(-1, -31)

In [50]:
torch.arange(start = 6, end = 1, step = 1)

RuntimeError: upper bound and larger bound inconsistent with step sign

In [44]:
kernel_northeast[0,0,range(kernel_size), [-1,-2,-3,-4,-5]]

tensor([0.0000, 0.0000, 0.3333, 0.0000, 0.0000])

In [28]:
kernel_northeast

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.5000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.3333, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]])

In [15]:
fill_kernel.flip(0)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3333, 0.3333, 0.3333,
        0.3333, 0.3333, 0.6667, 0.6667, 0.6667, 0.6667, 0.6667, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000])

In [10]:
kernel_northeast = (
            torch.zeros((1, 1, kernel_size, kernel_size)).type(
                torch.FloatTensor
))
kernel_northeast[0,0,range(30),range(30)] = fill_kernel

In [11]:
kernel_northeast

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0

In [6]:
# initialize the model and move to the computation device
model = MyModel(pretrained=create_model(NUM_CLASSES))
model = model.cuda()
# get the model parameters
params = [p for p in model.parameters() if p.requires_grad]
# define the optimizer
# optimizer = torch.optim.SGD(params, lr=0.0001, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(params, lr = 0.001)

# initialize the Averager class
train_loss_hist = Averager()
val_loss_hist = Averager()
train_itr = 1
val_itr = 1
# train and validation loss lists to store loss values of all...
# ... iterations till ena and plot graphs for all iterations
train_loss_list = []
val_loss_list = []
logits_list = []

# start the training epochs
for epoch in range(10):
    print(f"\nEPOCH {epoch+1} of {20}")

    # reset the training and validation loss histories for the current epoch
    train_loss_hist.reset()
    val_loss_hist.reset()
    
    # start timer and carry out training and validation
    start = time.time()
    train_loss = train(train_loader, model, optimizer, train_itr, train_loss_list, train_loss_hist)
    logits_list.append(list(next(model.parameters()).detach().cpu()))
    # print(next(model.parameters()).detach().cpu().numpy())
    val_loss = validate(valid_loader, model, val_itr, val_loss_list, val_loss_hist)
    print(f"Epoch #{epoch} train loss: {train_loss_hist.value:.3f}")
    print(f"Epoch #{epoch} validation loss: {val_loss_hist.value:.3f}")
    end = time.time()
    print(f"Took {((end - start) / 60):.3f} minutes for epoch {epoch}")


EPOCH 1 of 20
Training


Loss: 346.4468: 100%|██████████| 254/254 [00:41<00:00,  6.09it/s]  


Validating


Loss: 220.6905: 100%|██████████| 61/61 [00:02<00:00, 22.45it/s] 


Epoch #0 train loss: 748.444
Epoch #0 validation loss: 470.883
Took 0.742 minutes for epoch 0

EPOCH 2 of 20
Training


Loss: 1500.1703:  39%|███▉      | 100/254 [00:11<00:17,  8.90it/s]  


KeyboardInterrupt: 

In [16]:
import numpy as np
logits_array = np.array(logits_list)
logits_array[:,0]

array([0.462992  , 0.46298516, 0.46297824, 0.46297133, 0.46296442,
       0.4629575 , 0.4629506 , 0.46294367, 0.46293676, 0.46292984,
       0.46292293,        nan,        nan,        nan,        nan,
              nan,        nan,        nan,        nan,        nan],
      dtype=float32)